# Population Dynamics in CUDA V1



The simulation runs for `NUM_TIMEPERIODS`. At each time period all of the members of each species calls the `food_oracle` inorder for everyone to `gather_all_food`. After all food is collected we can `update_all_populations` based on the amount of food gathered. In order to do so we need to first `compute_local_population_share` which is the percentage of all species WITHIN A SINGLE REGION that belong to a given species. We can then use that to `update_community_population` for each community of each species based on 3 rules as listed in later sections of this document.

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Sun May 26 10:53:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpfxw6ytog".


In [4]:
%%cuda_group_save -n helpers.h -g default
#include <iostream>
#define NUM_REGIONS 4
#define NUM_SPECIES 4
#define COMMUNITIES_PER_NUM_SPECIES 4
#define COMMUNITIES_PER_REGION (NUM_SPECIES*COMMUNITIES_PER_NUM_SPECIES)
#define TOTAL_COMMUNITIES (NUM_REGIONS * COMMUNITIES_PER_REGION)
#define MAX_STARTING_POP 10
#define NUM_TIME_PERIODS 5
// for this simulation, we will only consider 1 block
#define NUM_BLOCKS 1
#define NUM_THREADS (TOTAL_COMMUNITIES/NUM_BLOCKS)
// We will have a block with 64 threads since we have 4 species, 4 communities per species, and 4 regions
//Now lets move on with the food oracle
typedef struct {
    int population; // the current count of population memebers
    int food_collected; // the current count of food collected
    int region_of_world; // region of the world we only have 2
    int species_type; // species type is just an int and we have 2 species
    float growth_rate; // this is a growth rate that we will define in latter functions
    float helperf; //this is our helper float

} Species_Community;
__host__ __device__
int gather_food(int community_id){ return 1; // returning 1 makes sure we get 1 unit per population member
    //random range integer in range [0, max_range]
}
__host__
int rand_range(int max_range){
    return rand() % max_range;
    //random float in range [0,1]
}
__host__
float rand01(){
    return (float)rand()/(float)RAND_MAX; // make sure to cas to float
}

In [5]:
%%cuda_group_save -n util.h -g default
#include "helpers.h"
//function to simulate population change for one community of one species
// 1) change in pop for a community is proportional to its growth rate and local population share
// 2) If the community has collected enough food to feed the population, it grows else it shrinks
// 3) If the population drops below 6, it goes extinct
__device__
void update_community_population(Species_Community *s_communities, int community_id, float local_population_share){
    float percent_change = local_population_share * s_communities[community_id].growth_rate;
    // proportional to means the derivative or rate of change is the product of the growth_rate and local_population_share
    int grow_or_shrink = s_communities[community_id].population >= s_communities[community_id].food_collected ? 1 : -1;
    int new_pop = s_communities[community_id].population * (grow_or_shrink * percent_change + 1); // new population
    s_communities[community_id].population = (new_pop >= 5) * new_pop;
}
// fctn to find the local population share
//1) Population share is defined as the percentage of pop in a region
//that is a given species across all communities of all species
// 2) We can assume this fctn is run as if it were a single thread.
__device__
float compute_local_population_share(Species_Community *s_communities, int community_id){
    //here we just want the total regional pop and the total species pop in the region
    int target_species = s_communities[community_id].species_type;
    int target_region = s_communities[community_id].region_of_world;
    int total_pop_in_the_region = 0;
    int total_species_pop_in_the_region = 0;
    for(int id =0; id < TOTAL_COMMUNITIES; id++){
        if(s_communities[id].region_of_world == target_region){
            total_pop_in_the_region += s_communities[id].population;
            if(s_communities[id].species_type == target_species){
                total_species_pop_in_the_region += s_communities[id].population;
            }
        }
    }
    __syncthreads();
    return ((float)total_species_pop_in_the_region)/((float)total_pop_in_the_region);
}

//fctn to simulate food gathering.
//1) total food is always reset to 0 for each call and each member of the population.
//2) Please use gather_food() to get a new amount of food for each memeber of the population
__device__
void gather_all_food(Species_Community *s_communities){
    //now
    for(int id = threadIdx.x; id < TOTAL_COMMUNITIES; id+=blockDim.x){
        //equivalently we can use id++ or id += blockDim.x bc our block dim is 1
        s_communities[id].food_collected = 0;
        //int population = (unsigned) s_communities[id].population;
        for(int i =0; i< s_communities[id].population; i++){
            s_communities[id].food_collected += gather_food(id);
        }

    }
    __syncthreads();
}

//update all populations for all communities of all species;
//1) We will need to use compute_ocal_population_share and update_community_population
//2) We can use as many threads as possible but the logic has to be safe
//3) We can use the helperf to store values when running the function in parallel
__device__
void update_all_populations(Species_Community *s_communities){
    for(int id = threadIdx.x; id < TOTAL_COMMUNITIES; id+=blockDim.x){
        s_communities[id].helperf = compute_local_population_share(s_communities, id);
    }
    __syncthreads();
    for(int id = threadIdx.x; id < TOTAL_COMMUNITIES; id+=blockDim.x){
        //printf("%u",blockDim.x);
        update_community_population(s_communities, id, s_communities[id].helperf);
    }
    __syncthreads();
}
__global__
void population_dynamics_kernel(Species_Community *d_communities){ //here we change the varname to d_communities since we run it in the device
    for(int i =0; i < NUM_TIME_PERIODS; i++){
        int community_id = threadIdx.x;
        if(community_id < TOTAL_COMMUNITIES){
            gather_all_food(d_communities);
        }
        __syncthreads();

        update_all_populations(d_communities);
    }
    //__syncthreads();

}

//The main functions

__host__
void population_dynamics(Species_Community *h_communities){
    //cudaSetDevice(0);
    Species_Community *d_communities;
    cudaMalloc(&d_communities, sizeof(Species_Community)*TOTAL_COMMUNITIES); //allocate data in the device // I confused this cuda_memcpy
    //below we are loading from host to device
    cudaMemcpy(d_communities, h_communities, sizeof(Species_Community)*TOTAL_COMMUNITIES, cudaMemcpyHostToDevice);
    cudaDeviceSynchronize();
    //run kernel
    population_dynamics_kernel<<<NUM_BLOCKS, NUM_THREADS>>>(d_communities);
    cudaDeviceSynchronize();
    cudaGetLastError(); //just in case to catch an error

    //now load from device to Host
    cudaMemcpy(h_communities, d_communities, sizeof(Species_Community) * TOTAL_COMMUNITIES, cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();
    //Now we free the memory
    cudaFree(d_communities);
}

In [6]:

%%cuda_group_save -n run.cu -g default
#include "util.h"
__host__
int main(){
    srand(1337);
    Species_Community h_communities[TOTAL_COMMUNITIES];
    for(int community_id =0; community_id < TOTAL_COMMUNITIES; community_id++){
        //I like to use id for the sake of simplicty but you can use community_id for clarity
        h_communities[community_id].population = rand_range(MAX_STARTING_POP) + 5; // set starting pop
        h_communities[community_id].region_of_world = rand_range(NUM_REGIONS); // we can randomly assign regions here; 0 or 1
        h_communities[community_id].species_type = rand_range(NUM_SPECIES); // species type is also random 0 or 1
        h_communities[community_id].growth_rate = rand01();  //between 0 and 1
    }
    for(int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
        std::cout << "ID[" << community_id << "]: of type [" << h_communities[community_id].species_type << "]: in region [" << h_communities[community_id].region_of_world <<
        "]: had initial population [" << h_communities[community_id].population << "]" << std::endl;
    }

    // now we call the main fctn for pop_dynamics
    population_dynamics(h_communities);

    //Now we can proceed to print the final results yay
    std::cout << "\n----------\n----------\n----------\n";

    for(int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
        std::cout << "ID[" << community_id << "]: of type [" << h_communities[community_id].species_type << "]: in region [" << h_communities[community_id].region_of_world <<
        "]: had final population [" << h_communities[community_id].population << "]" << std::endl;
    }
    return 0;


}

In [7]:
%cuda_group_run -g default

ID[0]: of type [3]: in region [2]: had initial population [6]
ID[1]: of type [1]: in region [2]: had initial population [9]
ID[2]: of type [3]: in region [3]: had initial population [13]
ID[3]: of type [0]: in region [3]: had initial population [9]
ID[4]: of type [2]: in region [0]: had initial population [10]
ID[5]: of type [0]: in region [0]: had initial population [5]
ID[6]: of type [2]: in region [2]: had initial population [5]
ID[7]: of type [3]: in region [3]: had initial population [12]
ID[8]: of type [1]: in region [2]: had initial population [12]
ID[9]: of type [0]: in region [2]: had initial population [14]
ID[10]: of type [0]: in region [3]: had initial population [10]
ID[11]: of type [1]: in region [0]: had initial population [11]
ID[12]: of type [2]: in region [0]: had initial population [9]
ID[13]: of type [1]: in region [2]: had initial population [9]
ID[14]: of type [2]: in region [3]: had initial population [9]
ID[15]: of type [3]: in region [2]: had initial population